# SQL


In [8]:
from MySQL import MySQL_Doing
import pandas as pd

MySQL_Doing = MySQL_Doing()
Results = MySQL_Doing.run("show tables")
df = pd.DataFrame(Results)
df

,Tables_in_bus_system
0,action_tour_hualien
1,admin_roles
2,admin_users
3,bus_route_stations
4,bus_routes
5,bus_routes_total
6,car_resource
7,login_state
8,reservation
9,reservation_record


In [31]:
df = pd.DataFrame(MySQL_Doing.run("select latitude , longitude from bus_route_stations where route_id = '1'"))
print(df.tail(1))
print("bus_route_stations")
df = pd.DataFrame(MySQL_Doing.select_all("bus_routes_total"))
print(df.tail(1))
print("bus_routes_total")
df = pd.DataFrame(MySQL_Doing.select_all("car_resource"))
print(df.tail(1))
print("car_resource")
df = pd.DataFrame(MySQL_Doing.run("select * from ttcarimport where car_licence = 'ABS-001'"))
print(df.tail(1))
print("ttcarimport")

    latitude   longitude
14  23.99302  121.603219
bus_route_stations
   route_id route_name  stop_count direction start_stop end_stop  status  \
4        14       逢甲大學           2        雙向       工109     工515       0   

           created_at  
4 2025-09-12 21:34:40  
bus_routes_total
   car_id car_licence  max_passengers car_status commission_date  \
6       7     ABS-003               5    service            None   

  last_service_date route  
6              None     3  
car_resource
      seq              rcv_dt car_licence             Gpstime          X  \
951  3813 2025-09-24 09:10:12     ABS-001 2025-09-24 09:10:12  23.992744   

              Y  Speed  Deg      acc  
951  121.619466     60   33  b'\x01'  
ttcarimport


# 1

In [71]:
last_stop_cache = {}

def update_bus_stop(route_id, car_licence, MySQL_Doing):
    # 撈最新座標
    car_pos = pd.DataFrame(MySQL_Doing.run(
        f"""
        SELECT X, Y FROM ttcarimport
        WHERE car_licence='{car_licence}'
        ORDER BY rcv_dt DESC LIMIT 1
        """
    ))
    if car_pos.empty:
        return None

    car_x, car_y = car_pos.iloc[0]["X"], car_pos.iloc[0]["Y"]

    # 撈該路線站點
    stops = pd.DataFrame(MySQL_Doing.run(
        f"""
        SELECT station_id, stop_order, latitude, longitude
        FROM bus_route_stations
        WHERE route_id={route_id}
        """
    ))

    # 找最近站
    stops["dist"] = stops.apply(
        lambda r: haversine(car_x, car_y, r["longitude"], r["latitude"]), axis=1
    )
    nearest = stops.sort_values("dist").iloc[0]
    curr_station = int(nearest["station_id"])   # 強制轉 int
    curr_order = int(nearest["stop_order"])

    # 上一次的站點
    prev_station = last_stop_cache.get(car_licence)

    direction = None
    if prev_station and prev_station != curr_station:
        prev_order = int(stops.loc[stops["station_id"] == prev_station, "stop_order"].iloc[0])
        if curr_order > prev_order:
            direction = "去程"
        else:
            direction = "回程"

    # 更新暫存
    last_stop_cache[car_licence] = curr_station

    return {"curr_station": curr_station, "direction": direction}


res = update_bus_stop(1, "NND-9607", MySQL_Doing)
print(res)
# e.g. {"curr_station": 27, "direction": None}


{'curr_station': 38, 'direction': None}


In [76]:
import math
import pandas as pd
import time
from datetime import datetime
import os

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 地球半徑 (公尺)
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

csv_path = "nearest_station_log.csv"
last_station = None
last_direction = None  # "去程" / "回程"

while True:
    # 1. 最新車輛座標
    car_df = pd.DataFrame(MySQL_Doing.run("""
        SELECT X, Y
        FROM ttcarimport
        WHERE car_licence = 'ABS-001'
        ORDER BY rcv_dt DESC
        LIMIT 1;
    """))
    car_lon = float(car_df.iloc[0]["X"])
    car_lat = float(car_df.iloc[0]["Y"])

    # 2. 站點座標
    stops_df = pd.DataFrame(MySQL_Doing.run("""
        SELECT station_id, stop_name, latitude, longitude, stop_order, direction
        FROM bus_route_stations
        WHERE route_id = '1';
    """))
    stops_df["latitude"] = stops_df["latitude"].astype(float)
    stops_df["longitude"] = stops_df["longitude"].astype(float)

    # 3. 計算距離
    stops_df["distance_m"] = stops_df.apply(
        lambda r: haversine(car_lat, car_lon, r["latitude"], r["longitude"]), axis=1
    )

    # 4. 找最近站
    nearest = stops_df.sort_values("distance_m").head(1).iloc[0]
    current_station = int(nearest["station_id"])
    current_order = int(nearest["stop_order"])

    # 5. 判斷方向
    direction = last_direction  # 預設沿用上一次
    if last_station is not None:
        last_order = int(stops_df.loc[stops_df["station_id"] == last_station, "stop_order"].values[0])
        if current_order > last_order:
            direction = "去程"
        elif current_order < last_order:
            direction = "回程"
        # 如果一樣，保持 last_direction 不變

    # 更新記錄
    last_station = current_station
    last_direction = direction

    # 6. 存檔
    log_row = {
        "timestamp": datetime.now(),
        "car_lat": car_lat,
        "car_lon": car_lon,
        "nearest_station": nearest["stop_name"],
        "station_id": current_station,
        "distance_m": nearest["distance_m"],
        "direction": direction or "未知"
    }
    df_log = pd.DataFrame([log_row])

    df_log.to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False)

    print(df_log)

    time.sleep(5)


                   timestamp   car_lat     car_lon nearest_station  \
0 2025-09-25 16:16:18.283131  23.99302  121.603219           花蓮轉運站   

   station_id  distance_m direction  
0          33         0.0        未知  
                   timestamp   car_lat     car_lon nearest_station  \
0 2025-09-25 16:16:23.327023  23.99302  121.603219           花蓮轉運站   

   station_id  distance_m direction  
0          33         0.0        未知  
                   timestamp    car_lat     car_lon nearest_station  \
0 2025-09-25 16:16:28.347374  23.988424  121.618438          婦幼親創園區   

   station_id  distance_m direction  
0          26         0.0        去程  
                   timestamp    car_lat     car_lon nearest_station  \
0 2025-09-25 16:16:33.403440  23.992912  121.617817          花蓮縣動防所   

   station_id  distance_m direction  
0          27         0.0        回程  
                   timestamp    car_lat     car_lon nearest_station  \
0 2025-09-25 16:16:38.448726  23.989522  121.619688      

KeyboardInterrupt: 

In [ ]:
# API 路由，使用 FastAPI
# @api.get("/GIS_About", tags=["Client"], summary="取得最新車輛資訊")
def get_gis_about():
    # 1. 從資料庫 car_resource 表格抓出全部資料，轉成 DataFrame
    df = pd.DataFrame(MySQL_Doing.select_all("car_resource"))
    
    # 2. 過濾掉 route 欄位為空的紀錄，只留下有路線的車輛
    df = df[df["route"].notnull()]

    # 3. 建立 {route: car_licence} 的字典，避免重複
    cars = {route: car for route, car in zip(df["route"], df["car_licence"])}
    carss = list(cars.values())  # 取出所有車牌號碼

    # 4. 如果沒有車輛，直接回傳空陣列
    if not carss:
        return []

    # 5. 把車牌號碼組成 SQL IN 查詢字串
    car_str = "', '".join(carss)

    # 6. SQL 查詢：取出每台車最新一筆資料 (seq 最大)
    query = f"""
    SELECT t.*
    FROM ttcarimport t
    INNER JOIN (
        SELECT car_licence, MAX(seq) AS max_seq
        FROM ttcarimport
        WHERE car_licence IN ('{car_str}')
        GROUP BY car_licence
    ) x ON t.car_licence = x.car_licence AND t.seq = x.max_seq;
    """

    # 7. 執行 SQL 查詢，取回結果並轉成 DataFrame
    df_latest = pd.DataFrame(MySQL_Doing.run(query))

    # 8. 建立車牌對應路線的 map，補上 route 欄位
    route_map = df.set_index("car_licence")["route"].to_dict()
    df_latest["route"] = df_latest["car_licence"].map(route_map)

    # 9. 把 DataFrame 轉成 dict，並把 NaN 改成 None，方便轉 JSON
    records = df_latest.where(pd.notnull(df_latest), None).to_dict(orient="records")

    # 10. 回傳 JSON 格式資料
    return records


# 測試呼叫 API
Results = get_gis_about()
df = pd.DataFrame(Results)
df

# Redis

In [9]:
import redis

r = redis.Redis(host='localhost', port=6379, db=0)

# 查詢所有 key
print(r.keys("*"))

# 查詢字串
print(r.get("mykey"))

# 查詢 hash
print(r.hgetall("user:1001"))


[b'session:eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YTBiNGZjOTciLCJleHAiOjE3NTg3MDQ5NzR9.uUo3gsRdcgcnE_9rmDoOMFMKH1YPzwOSlS1eP2uY_n0', b'line_tokens:Ufbcae02cac062d99d8bee745a0b4fc97', b'session:eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YTBiNGZjOTciLCJleHAiOjE3NTkyODk3NTB9.KyGXMLRr_fsxzGS25lUzXLNN2ikCshOIaZ_YB0rlzwQ', b'session:eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YTBiNGZjOTciLCJleHAiOjE3NTg4MDIxMzV9.w0MeEPUZCrvfv6-77AHUhojmHUoU47RLonLg-xYahvk', b'session:eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YTBiNGZjOTciLCJleHAiOjE3NTg3MDQ5NTh9.jO8Xk7qmod-XZR0yEc3JwmZUUyEH39QnoG3EIB-nyoc', b'session:eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YTBiNGZjOTciLCJleHAiOjE3NTg3MDcxOTB9.VOEBz96s5WcPjsRceXqmkL-O2Uqm2lvK2BHgEl-kigc', b'session:eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YTBiNGZjOTciLCJleHAiOjE3NTg3MDUwMTB9.oQCJ4rWWFsQrqHHEEZ3aiKmNDOymfjeO1_BWeeLS__Q', b'session:eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YTBiNGZjOTciLCJleHAiOjE3NTg3MTUwNTF9.zSpyHZYDotU8bFTXpPURifPNvAcP44cYTXs5

# Bus SQL


In [3]:
import pandas as pd
import pymysql

Infor = {
 'host': '43.212.245.102',
 'user': 'root',
 'port': 3306,
 'password': 'dnuiOKd217hdio',
 'database': 'Hualien_Bus',
 }

def init(Parameter):
    mydb = pymysql.connect(
        host=Parameter["host"],
        user=Parameter["user"],
        port=Parameter["port"],
        password=Parameter["password"],
        database=Parameter["database"],
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )
    return mydb

def MySQL_Run(query, Parameter=Infor):
    mydb = init(Parameter)
    mycursor = mydb.cursor()
    mycursor.execute(query)

    if query.strip().lower().startswith(("select", "show", "desc")):
        result = mycursor.fetchall()
    else:
        mydb.commit()
        result = "Query executed."

    mycursor.close()
    mydb.close()

    return (result)

MySQL_Run("""
show tables
""")
MySQL_Run("""
select * from ttcarimport
""")

()